In [177]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import re

In [150]:
pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [151]:
from sentence_transformers import SentenceTransformer
hugface_mdl = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')

In [152]:
mentions_df = pd.read_csv('mentions.csv', dtype=str)
print(f"Len(mentions_df)={len(mentions_df)}")
mentions_df.head(2)



Len(mentions_df)=9853


,idx,left_context,link_title,link_text,right_context,url,mention_in_page
0,0,стил е съвременният международно признат,светски,NaN,календар на който се основава,NaN,Григориански календар
1,1,е съвременният международно признат светски,календар,NaN,на който се основава и,NaN,Григориански календар


In [153]:
mentions_df['link_repr'] = mentions_df.apply(lambda x: x['link_title'] if x['link_text']is None else x['link_title'], axis=1)
mentions_df.head(2)

,idx,left_context,link_title,link_text,right_context,url,mention_in_page,link_repr
0,0,стил е съвременният международно признат,светски,NaN,календар на който се основава,NaN,Григориански календар,светски
1,1,е съвременният международно признат светски,календар,NaN,на който се основава и,NaN,Григориански календар,календар


In [178]:
entities_df = pd.read_csv('entities.csv')
print(f"Len(entities_df)={len(entities_df)}")
entities_df.head(2)

Len(entities_df)=501


,idx,title,text,url
0,0,Григориански календар,'Григорианският календар (понякога наричан и Г...,https://bg.wikipedia.org/wiki/%D0%93%D1%80%D0%...
1,1,GNU General Public License,GNU General Public License (на български преве...,https://bg.wikipedia.org/wiki/GNU_General_Publ...


In [155]:
# Merge mentions and entities (inner merge)
merge_df = mentions_df.merge(entities_df, \
                           left_on='link_title', \
                           right_on='title', \
                           how='inner',
                           suffixes=['_mention', '_entitity'])
print('Eligible mentions: ', len(merge_df))
merge_df.head(2)

Eligible mentions:  1512


,idx_mention,left_context,link_title,link_text,right_context,url_mention,mention_in_page,link_repr,idx_entitity,title,text,url_entitity
0,6,е въведен в употреба на,4 октомври,NaN,1582 г в съответствие с,NaN,Григориански календар,4 октомври,62,4 октомври,4 октомври е 277-ият ден в годината според гри...,https://bg.wikipedia.org/wiki/4_%D0%BE%D0%BA%D...
1,205,са следните За събитията до,4 октомври,NaN,1582 г включително има само,NaN,Приемане на григорианския календар,4 октомври,62,4 октомври,4 октомври е 277-ият ден в годината според гри...,https://bg.wikipedia.org/wiki/4_%D0%BE%D0%BA%D...


In [211]:
merge_df.reset_index(inplace=True)

In [156]:
class MentionEntityDataset(Dataset):
    def __init__(self, hugface_mdl, merge_df):
        self.mention_vecs = hugface_mdl.encode(pd.array( \
            merge_df['left_context'] +' ' +\
            merge_df['link_repr'] + ' ' + \
            merge_df['right_context']))
        self.entities_vec = hugface_mdl.encode(pd.array( \
            merge_df['title'] +' ' +\
            merge_df['text']))
        #cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        #output = cos(torch.from_numpy(self.mention_vecs), torch.from_numpy(self.entities_vec))
        #print("COS:", output)
        assert(len(self.mention_vecs) == len(self.entities_vec))

    def __len__(self):
        return len(self.mention_vecs)

    def __getitem__(self, idx):
        return self.mention_vecs[idx].reshape(-1), self.entities_vec[idx].reshape(-1)

In [157]:
dataset = MentionEntityDataset(hugface_mdl, merge_df)
len(dataset)

COS: tensor([0.5110, 0.5253, 0.6112,  ..., 0.6217, 0.6516, 0.7059])


1512

In [180]:
class MentionToEntityNet(nn.Module):
    def __init__(self, in_size=384, out_size=384):
        super(MentionToEntityNet, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_size, 2500),
            nn.ReLU(),
            nn.Linear(2500, 2500),
            nn.ReLU(),
            nn.Linear(2500, 2500),
            nn.ReLU(),
            nn.Linear(2500, out_size),
        )
        self.linear_relu_stack.apply(self._init_weights)
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [181]:
model = MentionToEntityNet()
print(model)

MentionToEntityNet(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=384, out_features=2500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2500, out_features=2500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=2500, out_features=2500, bias=True)
    (5): ReLU()
    (6): Linear(in_features=2500, out_features=384, bias=True)
  )
)


In [225]:
# Training cycle
MAX_EPOCHS = 100
BATCH_SIZE = 32
DISPLAY_STEP = 5
LEARNING_RATE = 0.001
def train(model, dataset):
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fct = nn.MSELoss()
    #loss_fct = nn.CosineSimilarity(eps=1e-6)
    model.train()
    for epoch in tqdm(range(1, MAX_EPOCHS+1)):
        losses = []
        dataloader = DataLoader(dataset, batch_size=BATCH_SIZE,
                        shuffle=True, drop_last=False)
        for x, y in dataloader:
            optimizer.zero_grad()
            logits = model(x)
            loss = loss_fct(logits, y)
            loss.backward()
            loss_value = loss.item()
            losses.append(loss_value)
        
        train_loss_value = np.mean(losses)
        
        # Display logs per each DISPLAY_STEP
        if (epoch) % DISPLAY_STEP == 0:
            print("Epoch: {:04d} loss={:.9f} ".format(epoch, train_loss_value))
        
    

train(model, dataset)
print ("Optimization Finished!")

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0005 loss=0.003360056 
Epoch: 0010 loss=0.003360078 
Epoch: 0015 loss=0.003359992 
Epoch: 0020 loss=0.003360340 
Epoch: 0025 loss=0.003359547 
Epoch: 0030 loss=0.003360750 
Epoch: 0035 loss=0.003360192 
Epoch: 0040 loss=0.003359968 
Epoch: 0045 loss=0.003359705 
Epoch: 0050 loss=0.003359662 
Epoch: 0055 loss=0.003359254 
Epoch: 0060 loss=0.003359984 
Epoch: 0065 loss=0.003359815 
Epoch: 0070 loss=0.003360312 
Epoch: 0075 loss=0.003360295 
Epoch: 0080 loss=0.003359157 
Epoch: 0085 loss=0.003360980 
Epoch: 0090 loss=0.003360356 
Epoch: 0095 loss=0.003359367 
Epoch: 0100 loss=0.003359574 
Optimization Finished!


# Evaluation

In [220]:
# https://bg.wikinews.org/wiki/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BD%D1%81%D0%B8%D1%80%D0%B0%D0%BD%D0%B5%D1%82%D0%BE_%D0%BD%D0%B0_%D0%A2%D0%B5%D0%B2%D0%B0%D1%82%D1%80%D0%BE%D0%BD_%D1%89%D0%B5_%D0%B1%D1%8A%D0%B4%D0%B5_%D0%BF%D1%80%D0%B5%D1%83%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%B5%D0%BD%D0%BE
text = """
Дни, използвани от слънчевите календари
При слънчевите календари датата отговаря 
на слънчевия ден. Денят може да се състои 
от периода между изгрев и залез слънце, 
последван от нощта, или може да представлява 
времето между повтарящи се събития, например 
два залеза. Дължината може да се променя малко 
през годината или може да се използва среден 
слънчев ден. Други типове календари също могат 
да използват слънчевия ден.

Юлиански и Григориански календар
Юлианският календар е въведен от римския диктатор 
Юлий Цезар през 46 г. пр.н.е. При него месеците са 
по-дълги от лунния цикъл и затова той не е удобен за 
следене на лунните фази, за сметка на това много точно 
показва сезоните. Обикновените години имат 365 дена, 
а всяка четвърта година е високосна, което означава, 
че има 366 дни. Така продължителността на средната 
година е 365,25 дни.
"""

In [221]:
text = re.sub(r"[,\\.\"]", r"", text.strip())
tokens = text.split()
tokens[0:5]

['Дни', 'използвани', 'от', 'слънчевите', 'календари']

In [226]:
model.eval()
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
MAX_SPAN_MENTION = 2
with torch.no_grad():
    for i in range(len(tokens)):
        for span in range(MAX_SPAN_MENTION): 
            left_tok_start = max(i+span-5,0)
            right_tok_end = min(i+span+5,len(tokens))
            #print(" ".join(tokens[left_tok_start: right_tok_end]))
            vec = hugface_mdl.encode(" ".join(tokens[left_tok_start: right_tok_end]))
            vec2 = model(torch.from_numpy(vec))
            output = cos(vec2.reshape(1,-1), torch.from_numpy(dataset.entities_vec))
            max_cos = torch.max(output).item()
            if max_cos > 0.15:
                print(torch.max(output))
                print(torch.argmax(output))
                print(tokens[i:i+span+1],'==>', merge_df['title'].iloc[torch.argmax(output).item()])
                #break
        
    

tensor(0.1561)
tensor(1231)
['ден', 'Денят'] ==> 8 януари
tensor(0.1561)
tensor(1231)
['Денят'] ==> 8 януари
